### **Loading Data**

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer



from prophet import Prophet

/Users/mustafaaldabbas/anaconda3/envs/myenv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
url1='https://raw.githubusercontent.com/data-bootcamp-v4/data/main/sales.csv'
#import dataset
df1 = pd.read_csv(url1)
df1.head()



,True_index,Store_ID,Day_of_week,Date,Nb_customers_on_day,Open,Promotion,State_holiday,School_holiday,Sales
0,0,625,3,2013-11-06,641,1,1,0,0,7293
1,1,293,2,2013-07-16,877,1,1,0,1,7060
2,2,39,4,2014-01-23,561,1,1,0,0,4565
3,3,676,4,2013-09-26,1584,1,1,0,0,6380
4,4,709,3,2014-01-22,1477,1,1,0,0,11647


In [3]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 640840 entries, 0 to 640839
Data columns (total 10 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   True_index           640840 non-null  int64 
 1   Store_ID             640840 non-null  int64 
 2   Day_of_week          640840 non-null  int64 
 3   Date                 640840 non-null  object
 4   Nb_customers_on_day  640840 non-null  int64 
 5   Open                 640840 non-null  int64 
 6   Promotion            640840 non-null  int64 
 7   State_holiday        640840 non-null  object
 8   School_holiday       640840 non-null  int64 
 9   Sales                640840 non-null  int64 
dtypes: int64(8), object(2)
memory usage: 48.9+ MB


In [27]:
url2='https://raw.githubusercontent.com/data-bootcamp-v4/data/main/ironkaggle_notarget.csv'
#import dataset
df2 = pd.read_csv(url2)
df2


,True_index,Store_ID,Day_of_week,Date,Nb_customers_on_day,Open,Promotion,State_holiday,School_holiday
0,7,764,4,2013-12-26,0,0,0,c,1
1,19,22,3,2013-05-22,449,1,0,0,1
2,31,1087,6,2013-06-29,622,1,0,0,0
3,45,139,6,2013-08-17,314,1,0,0,0
4,56,568,1,2014-04-07,356,1,0,0,0
...,...,...,...,...,...,...,...,...,...
71200,712004,217,2,2015-01-13,633,1,1,0,0
71201,712018,604,3,2014-04-30,743,1,1,0,0
71202,712020,1021,5,2014-07-18,1852,1,1,0,1
71203,712023,28,3,2014-08-27,0,0,0,0,1


### **1.Loading Data**
- 1.1 Load the Dataset: Import the dataset and understand its structure (columns, data types, etc.).
- 1.2 Handle Missing Values: Identify and handle missing values through imputation or removal.
- 1.3 Correct Data Types: Ensure that all columns have the correct data types (e.g., converting dates from strings to datetime objects).
- 1.4 Remove Duplicates: Identify and remove any duplicate rows in the dataset.
- 1.5 Handle Outliers: Detect and handle outliers, either by capping, transformation, or removal.

In [5]:
print(df1.columns)

Index(['True_index', 'Store_ID', 'Day_of_week', 'Date', 'Nb_customers_on_day',
       'Open', 'Promotion', 'State_holiday', 'School_holiday', 'Sales'],
      dtype='object')


In [6]:
# Convert Date column to datetime
df1['Date'] = pd.to_datetime(df1['Date'])

# Separate the Date column
date_column = df1['Date']
df1 = df1.drop(columns=['Date'])
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))])



In [7]:

# Identify numeric and categorical features excluding the target 'sales'
numeric_features = df1.drop(columns=['Sales']).select_dtypes(include=['int64', 'float64']).columns
categorical_features = df1.drop(columns=['Sales']).select_dtypes(include=['object']).columns

# Create transformers for numeric and categorical data
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),  # Impute missing values with the mean
    ('scaler', StandardScaler())])  # Scale numeric features

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),  # Impute missing values with the most frequent value
    ('encoder', OneHotEncoder(handle_unknown='ignore'))])  # One-hot encode categorical features

# Combine transformers into a preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

# Apply the transformations to the features (excluding 'sales')
X = preprocessor.fit_transform(df1.drop(columns=['Sales']))

# Get the target variable 'sales'
y = df1['Sales']

# Convert the result back to a DataFrame for the features
# Get the names of the transformed categorical columns
categorical_column_names = preprocessor.named_transformers_['cat']['encoder'].get_feature_names_out(categorical_features)

# Combine numeric and categorical columns
all_columns = np.hstack((numeric_features, categorical_column_names))
X = pd.DataFrame(X, columns=all_columns)

# Add back the Date column to the features if needed
X['Date'] = date_column.values

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


X

(512672, 12) (128168, 12) (512672,) (128168,)


,True_index,Store_ID,Day_of_week,Nb_customers_on_day,Open,Promotion,School_holiday,State_holiday_0,State_holiday_a,State_holiday_b,State_holiday_c,Date
0,-1.732010,0.207497,-0.500977,0.016379,0.452272,1.272689,-0.466094,1.0,0.0,0.0,0.0,2013-11-06
1,-1.732005,-0.823949,-1.001860,0.524897,0.452272,1.272689,2.145488,1.0,0.0,0.0,0.0,2013-07-16
2,-1.732000,-1.613067,-0.000095,-0.156000,0.452272,1.272689,-0.466094,1.0,0.0,0.0,0.0,2014-01-23
3,-1.731996,0.365942,-0.000095,2.048295,0.452272,1.272689,-0.466094,1.0,0.0,0.0,0.0,2013-09-26
4,-1.731991,0.468465,-0.500977,1.817738,0.452272,1.272689,-0.466094,1.0,0.0,0.0,0.0,2014-01-22
...,...,...,...,...,...,...,...,...,...,...,...,...
640835,1.732296,0.359728,1.001670,-0.048263,0.452272,-0.785738,-0.466094,1.0,0.0,0.0,0.0,2014-09-20
640836,1.732300,1.416028,-0.000095,1.365244,0.452272,1.272689,-0.466094,1.0,0.0,0.0,0.0,2015-01-15
640837,1.732305,-1.314818,1.001670,-0.082739,0.452272,-0.785738,-0.466094,1.0,0.0,0.0,0.0,2015-06-20
640838,1.732310,0.782248,-1.502742,-0.074120,0.452272,1.272689,2.145488,1.0,0.0,0.0,0.0,2014-08-18


### **train my models**

In [8]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Feature Engineering: Extracting useful features from the Date column
X_train['Year'] = X_train['Date'].dt.year
X_train['Month'] = X_train['Date'].dt.month
X_train['Day'] = X_train['Date'].dt.day
X_train['DayOfWeek'] = X_train['Date'].dt.dayofweek

X_test['Year'] = X_test['Date'].dt.year
X_test['Month'] = X_test['Date'].dt.month
X_test['Day'] = X_test['Date'].dt.day
X_test['DayOfWeek'] = X_test['Date'].dt.dayofweek

# Drop the Date column after feature engineering
X_train = X_train.drop(columns=['Date'])
X_test = X_test.drop(columns=['Date'])

# Initialize models
models = {
    "Linear Regression": LinearRegression(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest": RandomForestRegressor(),
    "XGBoost": XGBRegressor()
}

# Train and evaluate each model
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(f"{name} Performance:")
    print(f" - RMSE: {mean_squared_error(y_test, y_pred, squared=False):.4f}")
    print(f" - R^2 Score: {r2_score(y_test, y_pred):.4f}\n")


Linear Regression Performance:
 - RMSE: 1481.8118
 - R^2 Score: 0.8542

Decision Tree Performance:
 - RMSE: 1277.5256
 - R^2 Score: 0.8916

Random Forest Performance:
 - RMSE: 912.7466
 - R^2 Score: 0.9447

XGBoost Performance:
 - RMSE: 1008.1226
 - R^2 Score: 0.9325



### **Hyperparameter Tuning for RandomForestRegressor**

Skipped 

### **Train my model on the entire dataset**

In [12]:
# Combine the features and target variables
X_full = pd.concat([X_train, X_test], axis=0)
y_full = pd.concat([y_train, y_test], axis=0)


In [13]:
# Initialize the Random Forest model with the chosen or default parameters
rf_model = RandomForestRegressor(n_estimators=200, max_depth=20, random_state=42)

# Train the model on the entire dataset
rf_model.fit(X_full, y_full)


RandomForestRegressor(max_depth=20, n_estimators=200, random_state=42)

### **Preparing the second dataset for Prediction**

### **Loading Data**

In [18]:

# Convert Date column to datetime
df2['Date'] = pd.to_datetime(df2['Date'])

# Extract useful features from the Date column (if this was done in the first dataset)
df2['Year'] = df2['Date'].dt.year
df2['Month'] = df2['Date'].dt.month
df2['Day'] = df2['Date'].dt.day
df2['DayOfWeek'] = df2['Date'].dt.dayofweek

# Drop the original Date column if not needed
df2 = df2.drop(columns=['Date'])


In [20]:
# Apply the same preprocessing pipeline to the second dataset
df2_transformed = preprocessor.transform(df2)

# Convert the transformed data back to a DataFrame with the correct column names
df2_transformed = pd.DataFrame(df2_transformed, columns=all_columns)


In [21]:
df2_transformed

,True_index,Store_ID,Day_of_week,Nb_customers_on_day,Open,Promotion,School_holiday,State_holiday_0,State_holiday_a,State_holiday_b,State_holiday_c
0,-1.731976,0.639337,-0.000095,-1.364806,-2.211057,-0.785738,2.145488,0.0,0.0,0.0,1.0
1,-1.731918,-1.665882,-0.500977,-0.397330,0.452272,-0.785738,2.145488,1.0,0.0,0.0,0.0
2,-1.731859,1.642822,1.001670,-0.024561,0.452272,-0.785738,-0.466094,1.0,0.0,0.0,0.0
3,-1.731791,-1.302391,1.001670,-0.688219,0.452272,-0.785738,-0.466094,1.0,0.0,0.0,0.0
4,-1.731738,0.030411,-1.502742,-0.597721,0.452272,-0.785738,-0.466094,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
71200,1.732120,-1.060063,-1.001860,-0.000859,0.452272,1.272689,-0.466094,1.0,0.0,0.0,0.0
71201,1.732189,0.142255,-0.500977,0.236162,0.452272,1.272689,-0.466094,1.0,0.0,0.0,0.0
71202,1.732198,1.437775,0.500788,2.625764,0.452272,1.272689,2.145488,1.0,0.0,0.0,0.0
71203,1.732213,-1.647242,-0.500977,-1.364806,-2.211057,-0.785738,2.145488,1.0,0.0,0.0,0.0


### **Predict Sales in the second dataset**

In [25]:
# Get the expected columns from the training data
expected_columns = X_full.columns

# Ensure the second dataset has the same columns
df2_transformed = pd.DataFrame(df2_transformed, columns=all_columns)

# Reindex the second dataset to match the columns used during training
df2_transformed = df2_transformed.reindex(columns=expected_columns, fill_value=0)

# Now predict using the trained model
sales_predictions = rf_model.predict(df2_transformed)

# Add the predicted sales as a new column in the second dataset
df2['Predicted Sales'] = sales_predictions

# Save the updated DataFrame with predictions to a CSV file
df2.to_csv('df2_with_sales_predictions.csv', index=False)

print("Sales predictions have been successfully saved.")





Sales predictions have been successfully saved.


In [26]:
df2

,True_index,Store_ID,Day_of_week,Nb_customers_on_day,Open,Promotion,State_holiday,School_holiday,Year,Month,Day,DayOfWeek,Predicted Sales
0,7,764,4,0,0,0,c,1,2013,12,26,3,0.000000
1,19,22,3,449,1,0,0,1,2013,5,22,2,3730.829556
2,31,1087,6,622,1,0,0,0,2013,6,29,5,5717.936643
3,45,139,6,314,1,0,0,0,2013,8,17,5,3617.993730
4,56,568,1,356,1,0,0,0,2014,4,7,0,3851.124888
...,...,...,...,...,...,...,...,...,...,...,...,...,...
71200,712004,217,2,633,1,1,0,0,2015,1,13,1,6950.745639
71201,712018,604,3,743,1,1,0,0,2014,4,30,2,8881.729633
71202,712020,1021,5,1852,1,1,0,1,2014,7,18,4,17093.149684
71203,712023,28,3,0,0,0,0,1,2014,8,27,2,0.000000
